In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import numpy as np

import __init_path__
import env

from slam.models.relocalization.bovw import BoVW


font = {'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)


csv_path = '/home/d-zhukov/Projects/odometry/experiments/kitti_4_6/bovw_22/train/00/frame_history.csv'
dataset_root = '/datasets/odometry/KITTI_odometry_2012/'

In [ ]:
df = pd.read_csv(csv_path)
df.head()

In [ ]:
def visualize_with_kp(df, only_loops, difference_threshold):
    frame_total = df.to_index.iloc[-1]
    frame_counter = 1
    
    exctractor = cv2.xfeatures2d.SIFT_create()
    matcher = cv2.BFMatcher()
    
    while frame_counter <=  frame_total:  
        matches = df[df['to_index'] == frame_counter]
        frame_counter += 1
        
        if len(matches) > 0:
            
            if only_loops:
                max_difference = 0
                for k in range(len(matches)):
                    difference = matches['to_index'].iloc[k] - matches['from_index'].iloc[k]
                    if difference > max_difference:
                        max_difference = difference
                
                if max_difference < difference_threshold:
                    continue
            
            print(matches[['to_index', 'from_index']])
            
            image1 = cv2.imread(os.path.join(dataset_root, matches.to_path.iloc[0]))
            image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
            kp1, des1 = exctractor.detectAndCompute(np.uint8(image1), None)
            
            for k in range(3):
                if k < len(matches):
                    image2 = cv2.imread(os.path.join(dataset_root, matches.from_path.iloc[k]))
                    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
                else:
                    image2 = np.zeros_like(image1)
                kp2, des2 = exctractor.detectAndCompute(np.uint8(image2), None)
                if des1 is None or des2 is None:
                    continue
                
                bf_matches = matcher.knnMatch(des1, des2, 2)
                bf_matches = BoVW.ratio_test(bf_matches)
                print('Number of matches', len(bf_matches))
                bf_matches = [match[0] for match in bf_matches]
                image3 = cv2.drawMatches(image1, kp1, image2, kp2, bf_matches, None)
                plt.figure(figsize=(40, 20))
                plt.imshow(image3)
                plt.title(f'from {matches.from_index.iloc[k]} to {matches.to_index.iloc[k]}')
                plt.show()

In [ ]:
def visualize(df, only_loops, difference_threshold):
    frame_total = df.to_index.iloc[-1]
    frame_counter = 1
    
    while frame_counter <=  frame_total:  
        matches = df[df['to_index'] == frame_counter]
        frame_counter += 1
        
        if len(matches) > 0:
            
            if only_loops:
                max_difference = 0
                for k in range(len(matches)):
                    difference = matches['to_index'].iloc[k] - matches['from_index'].iloc[k]
                    if difference > max_difference:
                        max_difference = difference
                
                if max_difference < difference_threshold:
                    continue
            
            print(matches[['to_index', 'from_index']])
            
            image = cv2.imread(os.path.join(dataset_root, matches.to_path.iloc[0]))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            fig, axes = plt.subplots(2, 2, figsize=(40, 10))
            axes[0][0].title.set_text(f'to {matches.to_index.iloc[0]}')
            axes[0][0].imshow(image / 255)
            for k in range(3):
                if k < len(matches):
                    image = cv2.imread(os.path.join(dataset_root, matches.from_path.iloc[k]))
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    axes[(k + 1) // 2][(k + 1) % 2].title.set_text(f'from {matches.from_index.iloc[k]}')
                else:
                    image = np.zeros_like(image)
                    axes[(k + 1) // 2][(k + 1) % 2].title.set_text(f'No match')
                axes[(k + 1) // 2][(k + 1) % 2].imshow(image / 255)
            plt.show()

In [ ]:
is_keyframe = ~np.isnan(df['to_db_index'])
df = df[is_keyframe]

visualize(df, only_loops=True, difference_threshold=100)